# PaddleOCR-VL - Experimento com modelo multilíngue de OCR


---
---


Este notebook demonstra o uso do modelo **PaddleOCR-VL** para extração de texto de imagens através de OCR multilíngue

## Sobre o experimento

- **Modelo**: PaddlePaddle/PaddleOCR-VL
- **Objetivo**: Realizar OCR (Optical Character Recognition) e outras tarefas de reconhecimento em imagens e documentos
- **Ambiente**: Google Colab com GPU T4

## Configuração inicial

Primeiro, vamos instalar todas as dependências necessárias usando o `uv`


In [ ]:
# Install uv package manager
!pip install -q uv


### Instalação das dependências principais

Agora vamos instalar todas as bibliotecas necessárias:
- **PyTorch**: Framework de deep learning
- **Transformers**: Biblioteca da Hugging Face para modelos de linguagem
- **Pillow**: Para processamento de imagens


In [ ]:
# Install core dependencies with specific versions
!uv pip install transformers==4.57.1 pillow torch


---


## Importação das bibliotecas

Agora vamos importar todas as bibliotecas necessárias para o experimento


In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor
from PIL import Image

import torch
import time
import os


---


## Carregando o modelo

Vamos carregar o modelo **PaddleOCR-VL** e o processador correspondente


In [ ]:
# Device configuration
device = 'cuda'

# Model configuration
model_path = 'PaddlePaddle/PaddleOCR-VL'

print('=' * 70)
print('CARREGANDO MODELO PADDLEOCR-VL')
print(f'\nModelo: {model_path}')
print('=' * 70)

print('\nCarregando modelo...')
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code = True,
    torch_dtype = torch.bfloat16
).to(device).eval()
print('Modelo carregado com sucesso!')

print('\nCarregando processador...')
processor = AutoProcessor.from_pretrained(
    model_path,
    trust_remote_code = True
)
print('Processador carregado com sucesso!')

print('\n' + '=' * 70)
print(f'Dispositivo: {next(model.parameters()).device}')
print(f'Tipo de dados: {next(model.parameters()).dtype}')
print('=' * 70 + '\n')


---


## Configuração dos parâmetros

Vamos configurar os caminhos das imagens e os parametros para o processamento

### Tarefas disponíveis:

O PaddleOCR-VL suporta multiplas tarefas de reconhecimento:

| Tarefa        | Descrição                                 | Prompt                    |
|---------------|-------------------------------------------|---------------------------|
| **OCR**       | Reconhecimento óptico de caracteres       | `OCR:`                    |
| **Table**     | Reconhecimento de estrutura de tabelas    | `Table Recognition:`      |
| **Chart**     | Reconhecimento de gráficos e diagramas    | `Chart Recognition:`      |
| **Formula**   | Reconhecimento de fórmulas matemáticas    | `Formula Recognition:`    |

Neste experimento, vamos usar a tarefa **OCR** para extração de texto


In [ ]:
# Task configuration
chosen_task = 'ocr'  # Options: 'ocr' | 'table' | 'chart' | 'formula'

prompts = {
    'ocr': 'OCR:',
    'table': 'Table Recognition:',
    'formula': 'Formula Recognition:',
    'chart': 'Chart Recognition:',
}

# File paths
base_path = ''
image_path = os.path.join(base_path, 'data_experiments', 'case_01_drivers_license.jpeg')

print('CONFIGURAÇÃO DO EXPERIMENTO')
print('=' * 70)
print(f'Tarefa escolhida: {chosen_task.upper()}')
print(f'Prompt: {prompts[chosen_task]}')
print(f'\nArquivo de entrada: {image_path}')
print('=' * 70 + '\n')


---


## Processamento de OCR

Agora vamos processar a imagem e extrair o texto usando o modelo PaddleOCR-VL

In [ ]:
print('\n' + '=' * 70)
print('INICIANDO PROCESSAMENTO')
print('=' * 70 + '\n')

# Process the image
if not os.path.exists(image_path):
    print(f'Erro: arquivo de imagem não encontrado: {image_path}')
else:
    print(f'Processando imagem: {os.path.basename(image_path)}')
    print('-' * 70)

    try:
        # Load the image
        image = Image.open(image_path).convert('RGB')

        # Prepare messages for the model
        messages = [
            {
                'role': 'user',
                'content': [
                    {'type': 'image', 'image': image},
                    {'type': 'text', 'text': prompts[chosen_task]},
                ]
            }
        ]

        print('\nPreparando entrada para o modelo...')
        inputs = processor.apply_chat_template(
            messages,
            tokenize = True,
            add_generation_prompt = True,
            return_dict = True,
            return_tensors = 'pt'
        ).to(device)

        print('Executando inferência do modelo...')
        start_time = time.time()

        # Run inference
        outputs = model.generate(**inputs, max_new_tokens=1024)
        result = processor.batch_decode(outputs, skip_special_tokens=True)[0]

        end_time = time.time()
        inference_time = end_time - start_time

        print(f'Inferência concluida!')
        print(f'Tempo de processamento: {inference_time:.2f} segundos')

        print('\n' + '=' * 70)
        print('RESULTADO')
        print('=' * 70)
        print(result)
        print('=' * 70)

    except Exception as e:
        print(f'\nErro ao processar {os.path.basename(image_path)}: {str(e)}')
        import traceback
        traceback.print_exc()

print('\n' + '=' * 70)
print('PROCESSAMENTO CONCLUÍDO!')
print('=' * 70)


---


## Informações adicionais

### Sobre o modelo PaddleOCR-VL

O **PaddleOCR-VL** é um modelo de visão e linguagem desenvolvido pela PaddlePaddle capaz de realizar múltiplas tarefas de reconhecimento em documentos:

- **OCR multilíngue**: suporta reconhecimento de texto em multiplos idiomas
- **Reconhecimento de tabelas**: extrai estrutura e conteúdo de tabelas
- **Reconhecimento de fórmulas**: converte fórmulas matemáticas em LaTeX
- **Reconhecimento de gráficos**: analisa e extrai informações de gráficos e diagramas

### Tarefas suportadas

O modelo aceita diferentes prompts para diferentes tarefas:

1. **OCR**: `OCR:`
   - Extração de texto geral de imagens
   - Suporte multilíngue

2. **Table Recognition**: `Table Recognition:`
   - Reconhecimento de estrutura de tabelas
   - Extração de dados tabulares

3. **Formula Recognition**: `Formula Recognition:`
   - Conversão de fórmulas matemáticas para LaTeX
   - Suporta equações complexas

4. **Chart Recognition**: `Chart Recognition:`
   - Análise de gráficos gerais
   - Extração de dados e labels

### Configuração do modelo

- **max_new_tokens**: 1024 - número maximo de tokens gerados
- **torch_dtype**: bfloat16 - precisão brain float 16 para otimizar memória
- **trust_remote_code**: True - permite executar código customizado do modelo

### Requisitos recomendados de hardware

- GPU NVIDIA com CUDA
- Aproximadamente 8-10GB de VRAM
- Também pode rodar em CPU, mas será significativamente mais lento

### Vantagens do PaddleOCR-VL

- Modelo multilíngue com suporte a diversos idiomas
- Múltiplas tarefas em um único modelo
- Baseado em arquitetura Vision-Language moderna
- Boa precisão em documentos complexos
